## Python의 metaclasses 이해하기

**class 키워드를 사용할 때 python은 실행하며 객체를 만듬**  
**아래 코드는 메모리에 ObjectCreator라는 객체를 만듬**

In [2]:
class ObjectCreator:
    pass

my_object = ObjectCreator()
print(my_object)

In [3]:
print(ObjectCreator) # 클래스를 출력할 수 있습니다. 클래스가 객체이기 때문입니다.

def echo(o):
    print(o)

echo(ObjectCreator) # 클래스를 함수의 인자로 넘길 수 있습니다.

print(hasattr(ObjectCreator, 'new_attribute'))

ObjectCreator.new_attribute = 'foo' # 클래스에 새로운 속성을 추가할 수 있습니다.

print(hasattr(ObjectCreator, 'new_attribute'))

print(ObjectCreator.new_attribute)

ObjectCreatorMirror = ObjectCreator # 클래스를 변수에 할당할 수 있습니다.

print(ObjectCreatorMirror.new_attribute)

print(ObjectCreatorMirror())

<class '__main__.ObjectCreator'>
<class '__main__.ObjectCreator'>
False
True
foo
foo


**동적으로 클래스 생성하기**

###### class 키워드를 이용해 함수에서 클래스 만들기

In [4]:
def choose_class(name):
    if name == 'foo':
        class Foo(object):
            pass
        return Foo # 클래스 반환 (인스턴스 X)
    else:
        class Bar(object):
            pass
        return Bar

MyClass = choose_class('foo') 

print(MyClass) # 이 함수는 인스턴스가 아닌, 클래스를 반환합니다

print(MyClass()) # 이 클래스로 새로운 객체를 만들 수 있습니다

<class '__main__.choose_class.<locals>.Foo'>
<__main__.choose_class.<locals>.Foo object at 0x0000000005C6E5C0>


**type함수로 클래스만들기**  
type(name of the class,   
     tuple of the parent class (for inheritance, can be empty),   
     dictionary containing attributes names and values)  

In [6]:
class MyShinyClass(object):
        pass

# ↓ 와 같이 만들 수 있음

MyShinyClass = type('MyShinyClass', (), {}) # 클래스 객체 반환
print(MyShinyClass)

print(MyShinyClass()) # 클래스 인스턴스 생성


<class '__main__.MyShinyClass'>


In [7]:
class Foo(object):
    bar = True
    
# ↓ 와 같이 만들 수 있음

Foo = type('Foo', (), {'bar':True})

클래스와 동일하게 사용할 수 있음

In [8]:
print(Foo)

print(Foo.bar)

f = Foo()

print(f)

print(f.bar)


<class '__main__.Foo'>
True
True


상속도 할수 있음

In [10]:
class FooChild(Foo):
    pass

# ↓ 와 같이 만들 수 있음
FooChild = type('FooChild', (Foo,), {})
print(FooChild)

print(FooChild.bar) # bar is inherited from Foo


<class '__main__.FooChild'>
True


메소드도 추가할 수있음 (외부에 함수 생성하여)

In [12]:
def echo_bar(self):
    print(self.bar)

FooChild = type('FooChild', (Foo,), {'echo_bar': echo_bar})

print(hasattr(Foo, 'echo_bar'))

print(hasattr(FooChild, 'echo_bar'))

my_foo = FooChild()

my_foo.echo_bar()

False
True
True


일반 클래스 처럼 속성과 메소드를 추가 할 수도 있음

In [13]:
def echo_bar_more(self):
    print('yet another method')

FooChild.echo_bar_more = echo_bar_more
hasattr(FooChild, 'echo_bar_more')

True

## 최종: metaclasses는 무엇인가

메타클래스는 이러한 객체를 만드는 '무언가'입니다. 메타클래스는 '클래스'의 '클래스'이며, 다음과 같은 방법으로 묘사할 수 있습니다  

MyClass = MetaClass()  
MyObject = MyClass()  

In [16]:
#type 함수는 실제로 메타클래스
#type은 Python이 실제로 보이는 코드 뒤에서 클래스를 생성하는 메타클래스
MyClass = type('MyClass', (), {})

Python에서의 모든 것은 객체입니다.  
여기에는 정수, 문자열, 함수, 클래스를 포함합니다. 이 모든 것들은 객체입니다.  
그리고 이 모든 것들은 클래스로부터 생성됩니다

In [18]:
age = 35
print(age.__class__)

name = 'bob'
print(name.__class__)

def foo(): 
    pass

print(foo.__class__)

class Bar(object): 
    pass
b = Bar()
print(b.__class__)

<class 'int'>
<class 'str'>
<class 'function'>
<class '__main__.Bar'>


In [19]:
print(age.__class__.__class__)
print(name.__class__.__class__)
print(foo.__class__.__class__)
print(b.__class__.__class__)

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


**메타클래스는 그저 클래스 객체를 만드는 '무언가'라고 할 수 있음**

## __metaclass__ 속성

우리는 클래스 코드를 작성할 때 __metaclass__ 속성을 직접 추가할 수 있습니다.  
  
class Foo(object):  
  __metaclass__ = something...  
  
이렇게 하면 Python은 Foo 클래스를 생성하기 위해 직접 설정된 메타클래스를 사용하게 됩니다. 다만 약간의 주의가 필요합니다.  
  
우리는 class Foo(object) 코드를 먼저 작성했습니다, 하지만 Foo 클래스 객체는 메모리상에 아직 생성되지 않은 상태입니다.  
  
Python은 클래스 정의에 __metaclass__가 있는지 먼저 확인하게 될거고, 발견된 경우에 Foo 클래스를 만들기 위해 해당 메타클래스를 사용합니다. 발견하지 못한 경우에는 클래스를 만들기 위해 type을 사용하게 됩니다.  

In [20]:
class Foo(Bar):
    pass

Foo에 __metaclass__ 속성이 있나요?  
  
있으면, __metaclass__에 있는 걸로 Foo 클래스 객체(클래스 객체라고 말했습니다.)를 만듭니다.  
  
Python이 __metaclass__를 찾지 못했으면, Python은 __metaclass__를 모듈 레벨에서 찾고 위와 같은 방법으로 작동합니다. (단 상속받지 않은 클래스에 한해서만 그렇습니다, old-style classes 말이죠 - py2.)  
  
그래도 __metaclass__를 찾지 못했으면, Python은 Bar(가장 첫번째 부모 클래스)가 가진 메타클래스(여기서는 기본 메타클래스인 type)를 사용해서 클래스 객체를 만듭니다.

## 커스텀 메타클래스

In [25]:
# 메타클래스는 우리가 보통 `type`에 전달하는 객체와 같은 객체를 받습니다.
def upper_attr(future_class_name, future_class_parents, future_class_attr):

    """
    대문자로 변환된 속성의 리스트와 함께 클래스 객체를 반환합니다.
    """

    # '__'로 시작하지 않는 모든 객체를 가져와 대문자로 변환합니다.
    uppercase_attr = {}
    for name, val in future_class_attr.items():
        if not name.startswith('__'):
            uppercase_attr[name.upper()] = val
        else:
            uppercase_attr[name] = val

    # `type`으로 클래스를 생성합니다.
    return type(future_class_name, future_class_parents, uppercase_attr)


In [26]:
__metaclass__ = upper_attr # 이리하여 모듈 내에 있는 모든 클래스가 영향을 받게 됩니다.

In [36]:

#python 3.7
class Foo(metaclass=upper_attr):# 하지만 글로벌 메타클래스는 object와 함께 작동하지 않습니다
  # 하지만 우리는 이 클래스에만 영향을 주고자 여기에 __metaclss__를 정의하면
  # object 자식(children)과 함께 작동하게 됩니다.
    bar = 'bip'

print(hasattr(Foo, 'bar'))
# Out: False
print(hasattr(Foo, 'BAR'))
# Out: True

f = Foo()
print(f.BAR)
# Out: 'bip'

False
True
bip


이제 완전히 같은 코드를 작성하는데, 실제 클래스로 메타클래스를 작성해봅시다

In [37]:
# `type`이 실제로는 `str`, `int`와 같은 클래스라는 것을 기억해야 합니다.
# 따라서 우리는 `type`을 상속받을 수 있습니다.
class UpperAttrMetaclass(type): 
    # __new__는 __init__가 호출되기 전에 먼저 호출되는 메소드입니다.
    # 이 메소드는 실제 객체를 만들고 반환합니다.
    # __init__가 넘겨받은 인자로 객체를 초기화하는데 반해
    # 우리는 여기서 __new__를 사용합니다.
    # 여기서 생성되는 객체는 클래스이고 우리는 해당 클래스를 커스텀하기 위해 __new__를
    # 오버라이드해서 사용합니다.
    # 원하는 추가적인 행동에 대해서는 __init__에서 할 수 있습니다.
    # 몇몇 더 복잡한 사용 예에서는 __call__도 오버라이드해서 사용합니다
    # 하지만 이 예에서는 하지 않습니다.
    def __new__(upperattr_metaclass, future_class_name, 
                future_class_parents, future_class_attr):

        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return type(future_class_name, future_class_parents, uppercase_attr)

하지만 위 코드는 완전한 OOP 코드가 아닙니다. 우리는 type을 직접 호출했고 부모의 __new__ 코드를 오버라이드하거나 호출하지도 않았습니다.

In [38]:
class UpperAttrMetaclass(type): 

    def __new__(upperattr_metaclass, future_class_name, 
                future_class_parents, future_class_attr):

        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        # type.__new__ 메소드를 재사용합니다.
        # 이 예는 간단한 OOP 사용 예입니다, 흑마법은 없습니다.
        return type.__new__(upperattr_metaclass, future_class_name, 
                            future_class_parents, uppercase_attr)

 __new__는 항상 첫번째 인자로 정의된 클래스를 받습니다. 우리가 Python에서 클래스를 정의하고 클래스 메소드를 정의할 때 인스턴스를 받기 위해 첫번째 인자로 넣는 self와 같이 말이죠.

In [39]:
class UpperAttrMetaclass(type): 

    def __new__(cls, clsname, bases, dct):

        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return type.__new__(cls, clsname, bases, uppercase_attr)

우리는 위 코드를 간편하게 상속을 처리해주는 super를 이용해 더 깔끔하게 만들 수 있습니다

In [40]:
class UpperAttrMetaclass(type): 

    def __new__(cls, clsname, bases, dct):

        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return super(UpperAttrMetaclass, cls).__new__(cls, clsname, bases, uppercase_attr)